# Gene Cluster Family Analysis

Running BiG-SCAPE in a notebook in order to understand how it works ..

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sys
from glob import glob
import os
import pickle
from itertools import combinations
from array import array

sys.path.append('/Users/joewandy/git/BiG-SCAPE/')
sys.path.append('/Users/joewandy/anaconda/envs/bigscape/lib/python2.7/site-packages')

In [ ]:
import bigscape as bs
import functions as f
import ArrowerSVG as arr

from multiprocessing import Pool, cpu_count
from collections import defaultdict

## 1. Predicting domains using hmmscan

In [ ]:
output_folder = '/Users/joewandy/Dropbox/Meta_clustering/MS2LDA/BGC/data/bgcsforJustin/output'
f.create_directory(output_folder, "Output", False)
bgc_fasta_folder = os.path.join(output_folder, "fasta")
f.create_directory(bgc_fasta_folder, "BGC fastas", False)

In [ ]:
class bgc_data:
    def __init__(self, accession_id, description, product, records, max_width, organism, taxonomy, biosynthetic_genes,
                 contig_edge):
        # These two properties come from the genbank file:
        self.accession_id = accession_id
        self.description = description
        # AntiSMASH predicted class of compound:
        self.product = product
        # number of records in the genbank file (think of multi-locus BGCs):
        self.records = records
        # length of largest record (it will be used for ArrowerSVG):
        self.max_width = int(max_width)
        # organism
        self.organism = organism
        # taxonomy as a string (of comma-separated values)
        self.taxonomy = taxonomy
        # Internal set of tags corresponding to genes that AntiSMASH marked
        # as "Kind: Biosynthetic". It is formed as
        # clusterName + "_ORF" + cds_number + ":gid:" + gene_id + ":pid:" + protein_id + ":loc:" + gene_start + ":" + gene_end + ":strand:" + {+,-}
        self.biosynthetic_genes = biosynthetic_genes
        # AntiSMASH 4+ marks BGCs that sit on the edge of a contig
        self.contig_edge = contig_edge
        
bs.bgc_data = bgc_data
bs.mode = 'global'

In [ ]:
inputdir = '/Users/joewandy/Dropbox/Meta_clustering/MS2LDA/BGC/data/bgcsforJustin/'
bgc_info = {} # Stores, per BGC: predicted type, gbk Description, 
              # number of records, width of longest record, 
              # GenBank's accession, Biosynthetic Genes' ids

min_bgc_size = 0 # Provide the minimum size of a BGC to be included in the analysis. Default is 0 base pairs
exclude_gbk_str = '' # If this string occurs in the gbk filename, this file will not be used for the analysis

In [ ]:
# genbankDict: {cluster_name:[genbank_path_to_1st_instance,[sample_1,sample_2,...]]}
genbankDict = bs.get_gbk_files(inputdir, output_folder, bgc_fasta_folder, 
                            min_bgc_size, exclude_gbk_str, bgc_info)

# clusters and sampleDict contain the necessary structure for all-vs-all and sample analysis
clusters = genbankDict.keys()
clusterNames = tuple(sorted(clusters))
    
sampleDict = {} # {sampleName:set(bgc1,bgc2,...)}
gbk_files = [] # raw list of gbk file locations
for (cluster, (path, clusterSample)) in genbankDict.items():
    gbk_files.append(path)
    for sample in clusterSample:
        clustersInSample = sampleDict.get(sample, set())
        clustersInSample.add(cluster)
        sampleDict[sample] = clustersInSample
        
baseNames = set(clusters)

In [ ]:
allFastaFiles = set(glob(os.path.join(bgc_fasta_folder,"*.fasta")))
fastaFiles = set()
for name in baseNames:
    fastaFiles.add(os.path.join(bgc_fasta_folder, name+".fasta"))    
fastaBases = allFastaFiles.intersection(fastaFiles)
task_set = fastaFiles
verbose = False

In [ ]:
domtable_folder = os.path.join(output_folder, "domtable")
f.create_directory(domtable_folder, "Domtable", False)

pfam_dir = '/Users/joewandy/Downloads/pfam/'
# bs.pfam_dir = pfam_dir

In [ ]:
i = 1
for fastaFile in task_set:
    print 'Processing %d/%d' % (i, len(task_set))
    bs.runHmmScan(fastaFile, pfam_dir, domtable_folder, verbose)
    i += 1

## 2. Parse hmmscan domtable results and generate pfs and pfd files

In [ ]:
pfs_folder = os.path.join(output_folder, "pfs")
pfd_folder = os.path.join(output_folder, "pfd")    
f.create_directory(pfs_folder, "pfs", False)
f.create_directory(pfd_folder, "pfd", False)

In [ ]:
allDomtableFiles = set(glob(os.path.join(domtable_folder,"*.domtable")))
domtableFiles = set()
for name in baseNames:
    domtableFiles.add(os.path.join(domtable_folder, name+".domtable"))
domtableBases = allDomtableFiles.intersection(domtableFiles)
alreadyDone = set()

In [ ]:
bs.gbk_files = gbk_files
bs.genbankDict = genbankDict
bs.clusters = clusters
bs.baseNames = baseNames
bs.sampleDict = sampleDict

In [ ]:
# Specify at which overlap percentage domains are considered to overlap. 
# Domain with the best score is kept (default=0.1).
domain_overlap_cutoff = 0.1
for domtableFile in domtableFiles - alreadyDone:
    try:
        bs.parseHmmScan(domtableFile, pfd_folder, pfs_folder, domain_overlap_cutoff)
    except IndexError:
        continue
    except ValueError:
        continue

## 3. Parse the pfs, pfd files to generate BGC dictionary, clusters, and clusters per sample objects

Processing domain sequences files

In [ ]:
# allPfdFiles = set(glob(os.path.join(pfd_folder,"*.pfd")))
# pfdFiles = set()
# for name in baseNames:
#     pfdFiles.add(os.path.join(pfd_folder, name+".pfd"))
# pfdBases = allPfdFiles.intersection(pfdFiles)

## 4. Create SVG figures

In [ ]:
availableSVGs = set()
for svg in glob(os.path.join(svg_folder,"*.svg")):
    (root, ext) = os.path.splitext(svg)
    availableSVGs.add(root.split(os.sep)[-1])

In [ ]:
working_set = availableSVGs

In [ ]:
color_genes = arr.read_color_genes_file()
color_domains = arr.read_color_domains_file()
pfam_domain_categories = arr.read_pfam_domain_categories()

print("  Parsing hmm file for domain names")
pfam_info = {}
with open(os.path.join(pfam_dir, "Pfam-A.hmm"), "r") as pfam:
    putindict = False
    # assuming that the order of the information never changes
    for line in pfam:
        if line[:4] == "NAME":
            name = line.strip()[6:]
        if line[:3] == "ACC":
            acc = line.strip()[6:].split(".")[0]
        if line[:4] == "DESC":
            desc = line.strip()[6:]
            putindict = True

        if putindict:
            putindict = False
            pfam_info[acc] = (name, desc)
print("    Done")

#This must be done serially, because if a color for a gene/domain
# is not found, the text files with colors need to be updated
print("  Reading BGC information and writing SVG")
for bgc in working_set:
    arr.SVG(False, os.path.join(svg_folder,bgc+".svg"), genbankDict[bgc][0], os.path.join(pfd_folder,bgc+".pfd"), True, color_genes, color_domains, pfam_domain_categories, pfam_info, bgc_info[bgc].records, bgc_info[bgc].max_width)

color_genes.clear()
color_domains.clear()
pfam_domain_categories.clear()


## 5. Calculating distance matrix

Performing multiple alignment of domain sequences. First, Obtain all fasta files with domain sequences

In [ ]:
fasta_domains = set(glob(os.path.join(domains_folder,"*.fasta")))
print list(fasta_domains)[0]

Try to further reduce the set of domain fastas that need alignment

In [ ]:
fasta_domains_temp = fasta_domains.copy()
for domain_file in fasta_domains_temp:
    
    domain_name = ".".join(domain_file.split(os.sep)[-1].split(".")[:-1])

    # fill fasta_dict...
    with open(domain_file, "r") as fasta_handle:
        header_list = bs.get_fasta_keys(fasta_handle)

    # Get the BGC name from the sequence tag. The form of the tag is:
    # >BGCXXXXXXX_BGCXXXXXXX_ORF25:gid...
    sequence_tag_list = set(s.split("_ORF")[0] for s in header_list)

    # ...to find out how many sequences do we actually have
    if len(sequence_tag_list) == 1:
        
        # avoid multiple alignment if the domains all belong to the same BGC
        fasta_domains.remove(domain_file)
        print("Skipping Multiple Alignment for {} (appears only in one BGC)".format(domain_name))

In [ ]:
bs.pfam_dir = pfam_dir
i = 0
for domain in fasta_domains:
    print '%d/%d: %s' % (i, len(fasta_domains), domain)
    bs.run_hmmalign(domain)
    i += 1

Generating distance network files with ALL available input files

In [ ]:
networks_folder_all = "networks_all_hybrids"
bs.create_directory(os.path.join(output_folder, networks_folder_all), "Networks_all", False)

In [ ]:
bgc_class_weight = {}
bgc_class_weight["PKSI"] = (0.22, 0.76, 0.02, 1.0)
bgc_class_weight["PKSother"] = (0.0, 0.32, 0.68, 4.0)
bgc_class_weight["NRPS"] = (0.0, 1.0, 0.0, 4.0)
bgc_class_weight["RiPPs"] = (0.28, 0.71, 0.01, 1.0)
bgc_class_weight["Saccharides"] = (0.0, 0.0, 1.0, 1.0)
bgc_class_weight["Terpene"] = (0.2, 0.75, 0.05, 2.0)
bgc_class_weight["PKS-NRP_Hybrids"] = (0.0, 0.78, 0.22, 1.0)
bgc_class_weight["Others"] = (0.01, 0.97, 0.02, 4.0)

valid_classes = set()
for key in bgc_class_weight:
    valid_classes.add(key.lower())

bgc_class_weight["mix"] = (0.2, 0.75, 0.05, 2.0) # default when not separating in classes

BGC_classes = defaultdict(list)
# mix class will always be the last element of the tuple
bgcClassNames = tuple(sorted(list(bgc_class_weight)) + ["mix"])
assert bgcClassNames[-1] == 'mix'

bgcClassName2idx = dict(zip(bgcClassNames,range(len(bgcClassNames))))

In [ ]:
BGC_classes = defaultdict(list)
# bs.valid_classes = valid_classes

for clusterIdx,clusterName in enumerate(clusterNames):
    
    product = bgc_info[clusterName].product
    predicted_class = f.sort_bgc(product)
    if predicted_class.lower() in valid_classes:
        BGC_classes[predicted_class].append(clusterIdx)
        
    if predicted_class == "PKS-NRP_Hybrids":
        if "nrps" in valid_classes:
            BGC_classes["NRPS"].append(clusterIdx)
        if "t1pks" in product and "pksi" in valid_classes:
            BGC_classes["PKSI"].append(clusterIdx)
        if "t1pks" not in product and "pksother" in valid_classes:
            BGC_classes["PKSother"].append(clusterIdx)

    if predicted_class == "Others" and "-" in product:
        subclasses = set()
        for subproduct in product.split("-"):
            subclass = bs.sort_bgc(subproduct)
            if subclass.lower() in valid_classes:
                subclasses.add(subclass)
                
        # Prevent mixed BGCs with sub-Others annotations to get
        # added twice (e.g. indole-cf_fatty_acid has already gone
        # to Others at this point)
        if "Others" in subclasses:
            subclasses.remove("Others")

        for subclass in subclasses:
            BGC_classes[subclass].append(clusterIdx)
        subclasses.clear()

In [ ]:
cores = cpu_count()
bs.clusterNames = clusterNames
bs.bgcClassNames = bgcClassNames

In [ ]:
# Key: BGC. Item: ordered list of simple integers with the number of domains
# in each gene
# Instead of `DomainCountGene = defaultdict(list)`, let's try arrays of 
# unsigned ints
DomainCountGene = {}
# list of gene-numbers that have a hit in the anchor domain list. Zero based
corebiosynthetic_position = {}
# list of +/- orientation 
BGCGeneOrientation = {}

# if it's a re-run, the pfd/pfs files were not changed, so the skip_ma flag
# is activated. We have to open the pfd files to get the gene labels for
# each domain
# We now always have to have this data so the alignments are produced
for outputbase in baseNames:
    DomainCountGene[outputbase] = array('B')
    corebiosynthetic_position[outputbase] = array('B')
    BGCGeneOrientation[outputbase] = array('b')
    pfdFile = os.path.join(pfd_folder, outputbase + ".pfd")
    filtered_matrix = [map(lambda x: x.strip(), line.split('\t')) for line in open(pfdFile)]

    domain_counter = 0
    gene_number = 0
    gene_label = filtered_matrix[0][-1] # initialize with first label
    has_corebio = False
    for row in filtered_matrix:
        if row[-1] != gene_label:
            # we changed to a new gene. Check whether previous has a 
            # core biosynthetic / anchor domain hit
            if has_corebio:
                corebiosynthetic_position[outputbase].append(gene_number)
                has_corebio = False

            if gene_label[-1] == "+":
                BGCGeneOrientation[outputbase].append(1)
            else:
                BGCGeneOrientation[outputbase].append(-1)

            gene_label = row[-1] # update current label
            gene_number += 1 # advance gene number

            # record number of domains in previous gene
            DomainCountGene[outputbase].append(domain_counter)
            domain_counter = 1 # reset domain counter
        else:
            domain_counter += 1 # increase domain counter

        # TODO: if len(corebiosynthetic_position[outputbase]) == 0
        # do something with the list of pfam ids. Specifically, mark
        # (in this case TODO or always?) as biosynthetic genes, the ones that contain
        # domains from a special list. This list of special domains
        # comes from predicted domains within the CDSs marked as 'sec_met'
        # by antismash
        if row[-1] in bgc_info[outputbase].biosynthetic_genes:
            has_corebio = True

    # There is no transition when we finish, so analyze last gene
    if gene_label[-1] == "+":
        BGCGeneOrientation[outputbase].append(1)
    else:
        BGCGeneOrientation[outputbase].append(-1)
    DomainCountGene[outputbase].append(domain_counter)
    if has_corebio:
        corebiosynthetic_position[outputbase].append(gene_number)


In [ ]:
domains_folder = os.path.join(output_folder, "domains")
svg_folder = os.path.join(output_folder, "SVG")
f.create_directory(domains_folder, "Domains", False)
f.create_directory(svg_folder, "SVG", False)

BGCs = {} # will contain the BGCs
for outputbase in baseNames:
    
    print("Processing: " + outputbase)

    pfdFile = os.path.join(pfd_folder, outputbase + ".pfd")
    filtered_matrix = [[part.strip() for part in line.split('\t')] for line in open(pfdFile)]
    
    # save each domain sequence from a single BGC in its corresponding file
    fasta_file = os.path.join(bgc_fasta_folder, outputbase + ".fasta")
    with open(fasta_file, "r") as fasta_file_handle:
        fasta_dict = bs.fasta_parser(fasta_file_handle) # all fasta info from a BGC
    bs.save_domain_seqs(filtered_matrix, fasta_dict, domains_folder, outputbase)

    BGCs[outputbase] = f.BGC_dic_gen(filtered_matrix)
    del filtered_matrix[:]

    # store processed BGCs dictionary for future re-runs
    with open(os.path.join(output_folder, "BGCs.dict"), "wb") as BGC_file:
        pickle.dump(BGCs, BGC_file)
        BGC_file.close()
        
bs.BGCs = BGCs

In [ ]:
anchorfile = '/Users/joewandy/git/BiG-SCAPE/anchor_domains.txt'
anchor_domains = f.get_anchor_domains(anchorfile)
bs.anchor_domains = anchor_domains

Reading the ordered list of domains from the pfs files

In [ ]:
DomainList = {} # Key: BGC. Item: ordered list of domains
for outputbase in baseNames:
    pfsfile = os.path.join(pfs_folder, outputbase + ".pfs")
    if os.path.isfile(pfsfile):
        DomainList[outputbase] = f.get_domain_list(pfsfile)
    else:
        sys.exit(" Error: could not open " + outputbase + ".pfs")
        
bs.DomainList = DomainList
bs.DomainCountGene = DomainCountGene
bs.corebiosynthetic_position = corebiosynthetic_position
bs.BGCGeneOrientation = BGCGeneOrientation
bs.bgc_class_weight = bgc_class_weight
bs.domains_folder = domains_folder

In [ ]:
AlignedDomainSequences = {} # Key: specific domain sequence label. Item: aligned sequence
print(" Trying to read domain alignments (*.algn files)")
aligned_files_list = glob(os.path.join(domains_folder, "*.algn"))
if len(aligned_files_list) == 0:
    sys.exit("No aligned sequences found in the domain folder (run without the --skip_ma parameter or point to the correct output folder)")
for aligned_file in aligned_files_list:
    with open(aligned_file, "r") as aligned_file_handle:
        fasta_dict = f.fasta_parser(aligned_file_handle)
        for header in fasta_dict:
            AlignedDomainSequences[header] = fasta_dict[header]
bs.AlignedDomainSequences = AlignedDomainSequences

In [ ]:
# only make folders for the BGC_classes that are found
for bgc_class in BGC_classes:
    folder_name = bgc_class

    print("\n  {} ({} BGCs)".format(folder_name, str(len(BGC_classes[bgc_class]))))

    # create output directory   
    f.create_directory(os.path.join(output_folder, networks_folder_all, folder_name), "  All - " + bgc_class, False)

    # Create an additional file with the final list of all clusters in the class
    print("   Writing annotation files")
    path_list = os.path.join(output_folder, networks_folder_all, folder_name, "Network_Annotations_All_" + folder_name + ".tsv")
    with open(path_list, "w") as list_file:
        list_file.write("BGC\tAccesion ID\tDescription\tProduct Prediction\tBiG-SCAPE class\tOrganism\tTaxonomy\n")
        for idx in BGC_classes[bgc_class]:
            bgc = clusterNames[idx]
            product = bgc_info[bgc].product
            list_file.write("\t".join([bgc, bgc_info[bgc].accession_id, bgc_info[bgc].description, product, 
                                       f.sort_bgc(product), bgc_info[bgc].organism, bgc_info[bgc].taxonomy]) + "\n")

    if len(BGC_classes[bgc_class]) > 1:
        print("   Calculating all pairwise distances")
        pairs = set([tuple(sorted(combo)) for combo in combinations(BGC_classes[bgc_class], 2)])
        cluster_pairs = [(x, y, bgcClassName2idx[bgc_class]) for (x, y) in pairs]
        pairs.clear()
        network_matrix = bs.generate_network(cluster_pairs, cores)
        del cluster_pairs[:]

        print("   Writing output files")
        pathBase = os.path.join(output_folder, networks_folder_all, folder_name, "all" + folder_name)
        filenames = []
        for cutoff in cutoff_list:
            filenames.append("{}_c{:.2f}.network".format(pathBase, cutoff))
        cutoffs_and_filenames = list(zip(cutoff_list, filenames))
        del filenames[:]
        f.write_network_matrix(network_matrix, cutoffs_and_filenames, include_singletons, clusterNames, bgc_info)

        print("  Calling Gene Cluster Families")
        reduced_network = []
        for row in network_matrix:
            reduced_network.append([int(row[0]), int(row[1]), row[2]])
        del network_matrix[:]

        bs.clusterJsonBatch(BGC_classes[bgc_class], pathBase, reduced_network, cutoffs=cutoff_list,clusterClans=options.clans,clanCutoff=options.clan_cutoff)
        del BGC_classes[bgc_class][:]
        del reduced_network[:]